In [1]:
%matplotlib notebook
%matplotlib inline
import os

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
import pandas as pd
from collections import defaultdict

In [2]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
from PIL import Image
import os
import random

In [4]:
val_anno=load_json(os.path.join(OK_PATH,'mscoco_val2014_annotations.json'))['annotations']
ans_dict={}
for row in val_anno:
    ques_id=str(row['question_id'])
    answers=defaultdict(int)
    for info in row['answers']:
        answers[info['answer']]+=1
    #ans={ans:answers[ans]/sum(answers.values()) for ans in answers.keys()}
    ans_dict[ques_id]=answers
print (len(val_anno),len(ans_dict))

5046 5046


In [5]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [6]:
id_to_row={}
for row in vqa_all:
    id_to_row[str(row['question_id'])]=row

In [7]:
def get_score(occurences):
    return min(1.0,occurences/3.0)

import inflect
single = inflect.engine()

def norm_ans(ans):
    if ans.endswith(',') or ans.endswith('.'):
        ans=ans[:-1]
    if ans.startswith('a '):
        ans=ans[2:]
    if ',' in ans:
        ans=ans.split(',')[0]
    if len(ans.split(' '))==1 and ans not in ['grass','glass','bus']:
        norm_ans=single.singular_noun(ans)
        if norm_ans !=False:
            ans=norm_ans
    return ans

In [8]:
import nltk
from nltk.corpus.reader import VERB

_MANUAL_MATCHES = {
    "police": "police",
    "las": "las",
    "vegas": "vegas",
    "yes": "yes",
    "jeans": "jean",
    "hell's": "hell",
    "domino's": "domino",
    "morning": "morn",
    "clothes": "cloth",
    "are": "are",
    "riding": "ride",
    "leaves": "leaf",
    "dangerous": "danger",
    "clothing": "cloth",
    "texting": "text",
    "kiting": "kite",
    "firefighters": "firefight",
    "ties": "tie",
    "married": "married",
    "teething": "teeth",
    "gloves": "glove",
    "tennis": "tennis",
    "dining": "dine",
    "directions": "direct",
    "waves": "wave",
    "christmas": "christmas",
    "drives": "drive",
    "pudding": "pud",
    "coding": "code",
    "plating": "plate",
    "quantas": "quanta",
    "hornes": "horn",
    "graves": "grave",
    "mating": "mate",
    "paned": "pane",
    "alertness": "alert",
    "sunbathing": "sunbath",
    "tenning": "ten",
    "wetness": "wet",
    "urinating": "urine",
    "sickness": "sick",
    "braves": "brave",
    "firefighting": "firefight",
    "lenses": "lens",
    "reflections": "reflect",
    "backpackers": "backpack",
    "eatting": "eat",
    "designers": "design",
    "curiousity": "curious",
    "playfulness": "play",
    "blindness": "blind",
    "hawke": "hawk",
    "tomatoe": "tomato",
    "rodeoing": "rodeo",
    "brightness": "bright",
    "circuses": "circus",
    "skateboarders": "skateboard",
    "staring": "stare",
    "electronics": "electron",
    "electicity": "elect",
    "mountainous": "mountain",
    "socializing": "social",
    "hamburgers": "hamburg",
    "caves": "cave",
    "transitions": "transit",
    "wading": "wade",
    "creame": "cream",
    "toileting": "toilet",
    "sautee": "saute",
    "buildings": "build",
    "belongings": "belong",
    "stockings": "stock",
    "walle": "wall",
    "cumulis": "cumuli",
    "travelers": "travel",
    "conducter": "conduct",
    "browsing": "brows",
    "pooping": "poop",
    "haircutting": "haircut",
    "toppings": "top",
    "hearding": "heard",
    "sunblocker": "sunblock",
    "bases": "base",
    "markings": "mark",
    "mopeds": "mope",
    "kindergartener": "kindergarten",
    "pies": "pie",
    "scrapbooking": "scrapbook",
    "couponing": "coupon",
    "meetings": "meet",
    "elevators": "elev",
    "lowes": "low",
    "men's": "men",
    "childrens": "children",
    "shelves": "shelve",
    "paintings": "paint",
    "raines": "rain",
    "paring": "pare",
    "expressions": "express",
    "routes": "rout",
    "pease": "peas",
    "vastness": "vast",
    "awning": "awn",
    "boy's": "boy",
    "drunkenness": "drunken",
    "teasing": "teas",
    "conferences": "confer",
    "ripeness": "ripe",
    "suspenders": "suspend",
    "earnings": "earn",
    "reporters": "report",
    "kid's": "kid",
    "containers": "contain",
    "corgie": "corgi",
    "porche": "porch",
    "microwaves": "microwave",
    "batter's": "batter",
    "sadness": "sad",
    "apartments": "apart",
    "oxygenize": "oxygen",
    "striping": "stripe",
    "purring": "pure",
    "professionals": "profession",
    "piping": "pipe",
    "farmer's": "farmer",
    "potatoe": "potato",
    "emirates": "emir",
    "womens": "women",
    "veteran's": "veteran",
    "wilderness": "wilder",
    "propellers": "propel",
    "alpes": "alp",
    "charioteering": "chariot",
    "swining": "swine",
    "illness": "ill",
    "crepte": "crept",
    "adhesives": "adhesive",
    "regent's": "regent",
    "decorations": "decor",
    "rabbies": "rabbi",
    "overseas": "oversea",
    "travellers": "travel",
    "casings": "case",
    "smugness": "smug",
    "doves": "dove",
    "nationals": "nation",
    "mustange": "mustang",
    "ringe": "ring",
    "gondoliere": "gondolier",
    "vacationing": "vacate",
    "reminders": "remind",
    "baldness": "bald",
    "settings": "set",
    "glaced": "glace",
    "coniferous": "conifer",
    "revelations": "revel",
    "personals": "person",
    "daughter's": "daughter",
    "badness": "bad",
    "projections": "project",
    "polarizing": "polar",
    "vandalizers": "vandal",
    "minerals": "miner",
    "protesters": "protest",
    "controllers": "control",
    "weddings": "wed",
    "sometimes": "sometime",
    "earing": "ear",
}
_wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

import inflection
def stem(input_string):
    word_and_pos = nltk.pos_tag(nltk.tokenize.word_tokenize(input_string))
    stemmed_words = []
    for w, p in word_and_pos:
        if w in _MANUAL_MATCHES:
            w = _MANUAL_MATCHES[w]
        elif w.endswith("ing"):
            w = _wordnet_lemmatizer.lemmatize(w, VERB)
        elif p.startswith("NNS") or p.startswith("NNPS"):
            w = inflection.singularize(w)
        stemmed_words.append(w)
    return " ".join(stemmed_words) 

In [9]:
vis=0
acc=0.0
pred_file=load_pkl('pnp_ablations_65_10.pkl')
for ques_id in pred_file:
    pred=pred_file[ques_id]
    #pred=norm_ans(pred)
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
    vis+=1
print (acc*100.0/vis)
print (vis)

41.13489232395321
5046


In [10]:
vis=0
acc=0.0
for ques_id in pred_file:
    pred=pred_file[ques_id]
    pred=norm_ans(pred)
    pred=stem(pred)
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
    vis+=1
print (acc*100.0/vis)
print (vis)

44.53032104637353
5046


In [12]:
vis=0
acc=0.0
pred_file=load_pkl('pnp_ablations_65_0.pkl')
for ques_id in pred_file:
    pred=pred_file[ques_id]
    #pred=norm_ans(pred)
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
    vis+=1
print (acc*100.0/vis)
print (vis)

vis=0
acc=0.0
for ques_id in pred_file:
    pred=pred_file[ques_id]
    pred=norm_ans(pred)
    pred=stem(pred)
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
    vis+=1
print (acc*100.0/vis)
print (vis)

36.92693882943605
5046
39.63535473642512
5046
